In [ ]:
!pip3 install ortools pandas prettytable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 35.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [ ]:
import pandas as pd
import ortools
from ortools.sat.python import cp_model

data = pd.read_csv("courses.csv")
data = data.sample(frac=1).reset_index(drop=True)
courses = data.to_dict("records")

print(type(courses))

<class 'list'>


# Single Objective Approach

In [ ]:
# Construct model
model = cp_model.CpModel()

In [ ]:
# Decision Variable
x = dict()

for i in range(len(courses)):
  x[i] = model.NewBoolVar(f"course_{i}")

In [ ]:
print(x)

{0: course_0(0..1), 1: course_1(0..1), 2: course_2(0..1), 3: course_3(0..1), 4: course_4(0..1), 5: course_5(0..1), 6: course_6(0..1), 7: course_7(0..1), 8: course_8(0..1), 9: course_9(0..1), 10: course_10(0..1), 11: course_11(0..1), 12: course_12(0..1), 13: course_13(0..1), 14: course_14(0..1), 15: course_15(0..1), 16: course_16(0..1), 17: course_17(0..1), 18: course_18(0..1), 19: course_19(0..1), 20: course_20(0..1), 21: course_21(0..1), 22: course_22(0..1), 23: course_23(0..1), 24: course_24(0..1), 25: course_25(0..1), 26: course_26(0..1)}


In [ ]:
# Define Constraints
model.Add(sum(courses[i]["credit"] * x[i] for i in range(len(courses) )) == 180) # Total credit 180
model.Add(sum(courses[i]["credit"] * x[i] for i in range(len(courses)) if courses[i]["group"] == "CS") >= 120) # CS must be 120 credits
model.Add(sum(courses[i]["credit"] * x[i] for i in range(len(courses)) if courses[i]["exam_type"] == "EXAM") == 0) # Minimize Exam, assuming exam = 0

In [ ]:
total_cost = sum(courses[i]["cost"] * x[i] for i in range(len(courses)))
model.Minimize(total_cost)

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [ ]:
def format_table(_data):
  import prettytable
  from prettytable import PrettyTable

  table = PrettyTable()
  table.field_names = list(_data[0].keys())

  for row in _data:
    table.add_row(row.values())

  print(table)

In [ ]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution found!')

  _cost = 0
  _courses = list()

  for i in range(len(courses)):
    if solver.Value(x[i]) == 1:
      _courses.append(courses[i])
      _cost = _cost + courses[i]['cost']

  if status == cp_model.OPTIMAL:
    print(f"Optimal solution found, the total cost is {_cost}")
  if status == cp_model.FEASIBLE:
    print(f"Feasible solution found, the total cost is {_cost}")

  format_table(_courses)
else:
  print("No solution found")

Solution found!
Optimal solution found, the total cost is 12356
+-----------+-------+-----------+--------+------+
| course_id | group | exam_type | credit | cost |
+-----------+-------+-----------+--------+------+
|    M811   |   CS  |    EOMA   |   30   | 2245 |
|    M813   |   CS  |    EOMA   |   30   | 2245 |
|    D890   |  PROF |    EOMA   |   60   | 3781 |
|    S818   |   CS  |    EOMA   |   60   | 4085 |
+-----------+-------+-----------+--------+------+


# Multi Objective Approach

In [ ]:
# Construct model
model = cp_model.CpModel()

In [ ]:
# Decision Variable
x = dict()

for i in range(len(courses)):
  x[i] = model.NewBoolVar(f"course_{i}")

In [ ]:
# Define Constraints
model.Add(sum(courses[i]["credit"] * x[i] for i in range(len(courses) )) == 180) # Total credit 180
model.Add(sum(courses[i]["credit"] * x[i] for i in range(len(courses)) if courses[i]["group"] == "CS") >= 120) # CS must be 120 credits

In [ ]:
# Weight 0...1000
# SUM(1000)

weight_cost = 0.001
weight_exam = 0.999

In [ ]:
total_cost = sum(courses[i]["cost"] * x[i] for i in range(len(courses)))
total_exam = sum(x[i] for i in range(len(courses)) if courses[i]["exam_type"] == "EXAM")
model.Minimize((weight_cost * total_cost) + (weight_exam *total_exam))

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [ ]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print('Solution found!')

  _cost = 0
  _courses = list()

  for i in range(len(courses)):
    if solver.Value(x[i]) == 1:
      _courses.append(courses[i])
      _cost = _cost + courses[i]['cost']

  if status == cp_model.OPTIMAL:
    print(f"Optimal solution found, the total cost is {_cost}")
  if status == cp_model.FEASIBLE:
    print(f"Feasible solution found, the total cost is {_cost}")

  format_table(_courses)
else:
  print("No solution found")

Solution found!
Optimal solution found, the total cost is 12356
+-----------+-------+-----------+--------+------+
| course_id | group | exam_type | credit | cost |
+-----------+-------+-----------+--------+------+
|    M811   |   CS  |    EOMA   |   30   | 2245 |
|    M813   |   CS  |    EOMA   |   30   | 2245 |
|    D890   |  PROF |    EOMA   |   60   | 3781 |
|    S818   |   CS  |    EOMA   |   60   | 4085 |
+-----------+-------+-----------+--------+------+


Notes : Challenge 1 Course Selection